ETL Pipeline for Fortune 1000 Data
===

## Introduction

In [ ]:
'''
=================================================
ETL Pipeline for Fortune 1000 Data
Owner: Sam

This program will load the dataset and perform data validation using Great Expectations.
=================================================
'''

'\n=================================================\nMilestone 3\n\nNama  : Sam\nBatch : BSD-10\n\nThis program will load the dataset and perform data validation using Great Expectations.\n=================================================\n'

## Import library

In [2]:
import pandas as pd
import great_expectations as ge

## Data Loading

In [3]:
# Create a data context

from great_expectations.data_context import FileDataContext

context = FileDataContext.create(project_root_dir='.')

In [4]:
# Give a name to a Datasource. This name must be unique between Datasources.
datasource_name = 'fortune-1000-datasources'
datasource = context.sources.add_pandas(datasource_name)

# Give a name to a data asset
asset_name = 'fortune-1000'
path_to_data = 'P2M3_Sam_data_clean.csv'
asset = datasource.add_csv_asset(asset_name, filepath_or_buffer=path_to_data)

# Build batch request
batch_request = asset.build_batch_request()

In [5]:
# Creat an expectation suite
expectation_suite_name = 'expectation-clean-dataset'
context.add_or_update_expectation_suite(expectation_suite_name)

# Create a validator using above expectation suite
validator = context.get_validator(
    batch_request = batch_request,
    expectation_suite_name = expectation_suite_name
)

# Check the validator
validator.head()

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,company,rank,rank_change,revenue,profit,num_of_employees,sector,city,state,newcomer,ceo_founder,ceo_woman,profitable,prev_rank,ceo,website,ticker,market_cap
0,Walmart,1,0.0,572754.0,13673.0,2300000,Retailing,Bentonville,AR,no,no,no,yes,1,C. Douglas McMillon,https://www.stock.walmart.com,WMT,352037.0
1,Amazon,2,0.0,469822.0,33364.0,1608000,Retailing,Seattle,WA,no,no,no,yes,2,Andrew R. Jassy,www.amazon.com,AMZN,1202717.0
2,Apple,3,0.0,365817.0,94680.0,154000,Technology,Cupertino,CA,no,no,no,yes,3,Timothy D. Cook,www.apple.com,AAPL,2443962.0
3,CVS Health,4,0.0,292111.0,7910.0,258000,Health Care,Woonsocket,RI,no,no,yes,yes,4,Karen Lynch,https://www.cvshealth.com,CVS,125204.0
4,UnitedHealth Group,5,0.0,287597.0,17285.0,350000,Health Care,Minnetonka,MN,no,no,no,yes,5,Andrew P. Witty,www.unitedhealthgroup.com,UNH,500468.0


## Great Expectations

### Expectation 1: Company names should be unique

In [6]:
# Expectation 1
validator.expect_column_values_to_be_unique(column="company")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 948,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 2: Revenue should be between reasonable values (0 and 1 trillion)

In [7]:
# Expectation 2
validator.expect_column_values_to_be_between(
column="revenue",
min_value=0,
max_value=10000000,
mostly=0.95
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 948,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 3: Sector values should be in a known set

In [ ]:
# Expectation 3
valid_sectors = [
"Retailing", "Technology", "Health Care", "Energy", "Financials",
"Food & Drug Stores", "Motor Vehicles & Parts", "Telecommunications",
"Transportation", "Aerospace & Defense", "Foshod, Beverages & Tobacco",
"Household Products", "Media", "Chemicals", "Materials", "Industrials",
"Apparel", "Hotels, Restaurants & Leisure", "Wholesalers", "Engineering & Construction",
"Business Services"
]
validator.expect_column_values_to_be_in_set("sector", valid_sectors)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 948,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 4: Profit column should be numeric

In [9]:
# Expectation 4
validator.expect_column_values_to_be_of_type("profit", "float64")

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": "float64"
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 5: Validate CEO gender consistency when CEO is female

In [10]:
validator.expect_column_values_to_be_in_set(
    column="ceo_woman",
    value_set=["yes", "no"],
    condition_parser="pandas",
    row_condition='ceo_woman == "yes"'
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 73,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 6: Validate that profitable companies have positive profit values

In [11]:
# Expectation 6
validator.expect_column_values_to_be_between(
    column="profit",
    min_value=0,
    max_value=None,
    condition_parser="pandas",
    row_condition='profitable=="yes"'
)

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 841,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

### Expectation 7: Custom expectation - Market capitalization should not be null (3rd party expectation)

In [12]:
# Expectation 7
validator.expect_column_values_to_not_be_null(column="market_cap")

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 948,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}